In [1]:
import numpy as np
import pickle
import os
import io
import torch
from joints import MAIN_JOINTS
from typing import List

In [2]:
class CPU_Unpickler(pickle.Unpickler):
  def find_class(self, module, name): 
    if module == 'torch.storage' and name == '_load_from_bytes':
      return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
    else:
      return super().find_class(module, name )

def read_pickle(path):
    with open(path, 'rb') as f:
      return CPU_Unpickler(f).load()

In [3]:
def reshape_joints(input_array: np.array) -> np.array:
    """
    Reshape an ND array of shape (N, 84) to a (28, 3) array corresponding to joint positions.
    
    Parameters:
        input_array (np.ndarray): A 1D NumPy array of length 84.
        
    Returns:
        np.ndarray: A reshaped (28, 3) array.

    """
    
    length = input_array.shape[0] if input_array.shape != (84,) else 1
    return input_array.reshape(length, 28, 3)


In [4]:
joint_names = [
    'pelvis', 'left_hip', 'right_hip',
    'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist',
    'head', 'jaw', 'nose', 'right_eye', 'left_eye', 'right_ear', 'left_ear',
    'left_shoulder', 'right_shoulder', 'left_collar', 'right_collar', 'neck',
    'spine1', 'spine2', 'spine3',
    'left_knee', 'right_knee', 'left_ankle', 'right_ankle', 'left_foot', 'right_foot'
]

spatial_edges = [
    # Head cluster
    ('left_eye', 'head'), ('right_eye', 'head'), ('left_ear', 'head'),
    ('right_ear', 'head'), ('nose', 'head'), ('jaw', 'head'),

    # Left arm
    ('left_wrist', 'left_elbow'), ('left_elbow', 'left_shoulder'),
    ('left_shoulder', 'left_collar'),

    # Right arm
    ('right_wrist', 'right_elbow'), ('right_elbow', 'right_shoulder'),
    ('right_shoulder', 'right_collar'),

    # Spine
    ('spine1', 'spine2'), ('spine2', 'spine3'),

    # Pelvis/hip
    ('right_hip', 'pelvis'), ('left_hip', 'pelvis'), ('pelvis', 'spine3'),

    # Legs
    ('right_foot', 'right_ankle'), ('right_ankle', 'right_knee'), ('right_knee', 'right_hip'),
    ('left_foot', 'left_ankle'), ('left_ankle', 'left_knee'), ('left_knee', 'left_hip'),

    # Shoulder to spine
    ('right_collar', 'spine1'), ('left_collar', 'spine1'),

    # Head to spine
    ('head', 'spine1'),
]


def get_filtered_joint_list(exclude_groups: List) -> List:
    joint_groups = {
        "pelvic_joints": ['pelvis', 'left_hip', 'right_hip'],
        "arm_joints": ['left_elbow','right_elbow','left_wrist','right_wrist'],
        "head_joints": ['head','jaw','nose','right_eye','left_eye','right_ear','left_ear'],
        "thorax_joints": ['left_shoulder','right_shoulder','left_collar','right_collar','neck'],
        "leg_joints": ['left_knee','right_knee','left_ankle','right_ankle','left_foot','right_foot'],
        "spine_joints": ['spine1', 'spine2', 'spine3']
    }

    # Flatten group names into a set of excluded joint names
    excluded = set()
    for group in exclude_groups:
        excluded.update(joint_groups[group])

    # Final joint list after filtering
    final_joint_list = [j for j in joint_names if j not in excluded]
    return final_joint_list

def filter_edges(joint_list: List, spatial_edges: List) -> List:
    joint_set = set(joint_list)
    filtered_edges = [(a, b) for a, b in spatial_edges if a in joint_set and b in joint_set]
    return filtered_edges

def build_edge_list(joint_list: List, spatial_edges: List, num_frames=150):
    joint_idx = {name: i for i, name in enumerate(joint_list)}
    N = len(joint_list)
    total_nodes = N * num_frames

    rows, cols = [], []

    for t in range(num_frames):
        offset = t * N

        # spatial connections within the frame
        for a, b in spatial_edges:
            i, j = joint_idx[a] + offset, joint_idx[b] + offset
            rows += [i, j]
            cols += [j, i]

        # temporal connections between same joints across frames
        if t < num_frames - 1:
            next_offset = (t + 1) * N
            for i in range(N):
                rows += [offset + i, next_offset + i]
                cols += [next_offset + i, offset + i]

    return torch.tensor([rows, cols], dtype = torch.long)

In [5]:
# Test 
included_joints = get_filtered_joint_list(exclude_groups=[]) 
filtered_edges = filter_edges(included_joints, spatial_edges)
adj = build_edge_list(included_joints, filtered_edges, num_frames=150)
for i in range(28):
    print((adj == i).any())
adj

tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)


tensor([[  11,    7,   10,  ..., 4191, 4179, 4191],
        [   7,   11,    7,  ..., 4188, 4191, 4179]])

In [6]:
def build_node_list(exluded_groups : List, frames: np.array) -> np.array:
     joints_list = get_filtered_joint_list(exluded_groups)
     joint_indices = [MAIN_JOINTS.index(joint) for joint in joints_list]
     reshaped_frames = reshape_joints(frames)
     return reshaped_frames[:, joint_indices]
     

In [11]:
#test
frames = read_pickle("action_dataset_joints_leg_sampled_150.pkl")
frames['train'][0]
data = build_node_list([], frames['train'][0][0])
reshaped_frame = reshape_joints(frames['train'][0][0])

#should be true
print(reshaped_frame.shape == data.shape)
print(data.shape)

#should have 3 less joints
data = build_node_list(['pelvic_joints'], frames['train'][0][0])
print(data.shape)


print(type(frames['train'][0][0]))

True
(150, 28, 3)
(150, 25, 3)
<class 'numpy.ndarray'>


/var/folders/z9/hg_4x2rn2t52_93qz6qk1_gh0000gn/T/ipykernel_47173/1925829092.py:6: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  return super().find_class(module, name )
